In [1]:
!pip install -q git+https://github.com/PeptoneLtd/nerfax.git

     |████████████████████████████████| 2.1 MB 9.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 270 kB 49.4 MB/s 
     |████████████████████████████████| 2.6 MB 37.8 MB/s 


In [2]:
import nerfax
import mdtraj as md

path = 'model.pdb'
!wget https://alphafold.ebi.ac.uk/files/AF-Q93WI9-F1-model_v4.pdb -q -O {path}

# Load pdb file with mdtraj into scnet internal coordinate format
internal_coords = nerfax.parser.load_pdb(path)
# Reconstruct scnet cartesian (L,14,3) from internal coordinate format
reconstructed_cartesian_scnet_coords = nerfax.plugin.protein_fold(**internal_coords)

# Copy the resulting scnet coords back into 'normal' mdtraj format
t = nerfax.parser.load_traj(path)
_, _restrict_to_scnet_atoms, _scnet_to_list = nerfax.parser.get_scnet_loader_fns(t)
t = _restrict_to_scnet_atoms(t)
t.xyz = _scnet_to_list(reconstructed_cartesian_scnet_coords)[None]

# Check RMSD to original positions
t_pre = _restrict_to_scnet_atoms(nerfax.parser.load_traj(path))
print(f'RMSD between reconstructed and original : {md.rmsd(t,t_pre)[0]:.1g}')

/usr/local/lib/python3.7/dist-packages/mdtraj/formats/pdb/pdbfile.py:202: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  'cell vectors.', category=UserWarning)
/usr/local/lib/python3.7/dist-packages/mdtraj/formats/pdb/pdbfile.py:202: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  'cell vectors.', category=UserWarning)


RMSD between reconstructed and original : 0.0006


If the reconstruction is to be done on the same protein many times over we can compile the reconstruction, specialised to that exact protein

In [3]:
from jax import jit, block_until_ready
fold = jit(nerfax.plugin.get_jax_protein_fold(internal_coords))
inputs = {k:internal_coords[k] for k in ['angles_mask', 'bond_mask']}
# compile
_ = fold(**inputs)
# time
%time output = block_until_ready(fold(**inputs))

CPU times: user 558 µs, sys: 0 ns, total: 558 µs
Wall time: 582 µs


The fold by default uses the associative scan, however this is not optimal for running on CPU. We can switch to using the sequential version instead and should get a faster benchmark

In [4]:
from functools import partial
fold = jit(nerfax.plugin.get_jax_protein_fold(internal_coords, 
                  reconstruct_fn=partial(nerfax.reconstruct.reconstruct_from_internal_coordinates, mode='sequential')
          ))
# compile
_ = fold(**inputs)
# time
%time output = block_until_ready(fold(**inputs))

CPU times: user 413 µs, sys: 0 ns, total: 413 µs
Wall time: 423 µs
